In [2]:
import torchosr

/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
from torchvision import transforms
data = torchosr.data.base_datasets.MNIST_base(root='data', 
                                              download=True, 
                                              transform=transforms.Compose([
                                                transforms.Resize(28),
                                                transforms.ToTensor()]))
data

Dataset MNIST_base
    Number of datapoints: 70000
    Root location: data
    StandardTransform
Transform: Compose(
               Resize(size=28, interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )

In [22]:
config, openness = torchosr.data.configure_division(data, 
                                                    n_openness=3, 
                                                    repeats=3, 
                                                    seed=1234)

for i, (kkc, uuc) in enumerate(config):
    print('C%i - Op: %.3f KKC:%s \t UUC:%s' % (
        i, 
        openness[int(i/3)].detach().numpy(), 
        kkc.detach().numpy(), 
        uuc.detach().numpy()))

C0 - Op: 0.047 KKC:[0 1 7 9 3] 	 UUC:[6]
C1 - Op: 0.047 KKC:[6 4 9 2 7] 	 UUC:[1]
C2 - Op: 0.047 KKC:[1 6 7 0 5] 	 UUC:[9]
C3 - Op: 0.225 KKC:[8 4 5] 	 UUC:[3 1 9 6]
C4 - Op: 0.225 KKC:[9 7 4] 	 UUC:[2 5 0 8]
C5 - Op: 0.225 KKC:[0 4 2] 	 UUC:[3 9 6 8]
C6 - Op: 0.397 KKC:[3 6] 	 UUC:[9 1 4 0 8 7 5]
C7 - Op: 0.397 KKC:[2 5] 	 UUC:[9 8 6 3 1 4 7]
C8 - Op: 0.397 KKC:[4 1] 	 UUC:[3 0 5 9 2 7 8]


In [31]:
from torch.utils.data import DataLoader

kkc, uuc = config[0]
train_data, test_data = torchosr.data.get_train_test(data,
                                                   kkc, uuc,
                                                   root = 'data',
                                                   tunning = False,
                                                   fold = 0,
                                                   seed = 1234,
                                                   n_folds = 5)

train_data_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=64, shuffle=True)



In [35]:
import numpy as np

X, y = next(iter(train_data_loader))
print('Train labels:', np.unique(np.argmax(y, axis=1)))
    
X, y = next(iter(test_data_loader))
print('Test labels:', np.unique(np.argmax(y, axis=1)))


Train labels: [0 1 2 3 4]
Test labels: [0 1 2 3 4 5]


In [36]:
methods = [
            torchosr.models.TSoftmax(lower_stack=torchosr.architectures.fc_lower_stack(depth=1, img_size_x=28, n_out_channels=64),
                                n_known=len(kkc),
                                epsilon=torchosr.utils.get_softmax_epsilon(len(kkc))),
            torchosr.models.Openmax(lower_stack=torchosr.architectures.fc_lower_stack(depth=1, img_size_x=28, n_out_channels=64),
                               n_known=len(kkc),
                               epsilon=torchosr.utils.get_openmax_epsilon(len(kkc))),
            ]

AttributeError: module 'torchosr.utils' has no attribute 'get_softmax_epsilon'